# Download Torrent to Google Drive

### 1. Install libtorrent Dependency

In [ ]:
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

### 2. Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

## 3. Add Torrent Source



### [Option #1]

In [ ]:
# @title Upload .torrent file

# Upload a torrent file
from google.colab import files

source = files.upload()
info = lt.torrent_info(list(source.keys())[0])
h = ses.add_torrent({'ti': info, 'save_path': '/content/drive/MyDrive/Torrents'})

print('starting', h.status().name)

### [Option #2]

In [ ]:
# @title Enter Magnet Link Or Type Exit: exit

# Add torrents from multiple magnet link
downloads = []

while True:  
    magnet_link = input("Enter a magnet link or type 'exit' to stop: ")
    
    if magnet_link.lower() == 'exit':
        break
    
    params = lt.parse_magnet_uri(magnet_link)
    params.save_path = '/content/drive/MyDrive/Torrents'
    h = ses.add_torrent(params)
    downloads.append(h)
    
    print('\nstarting', h.status().name)

## 4. Download File

In [ ]:
import time
import sys

for h in downloads:
    while (not h.status().is_seeding):
        s = h.status()

        print('\r%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s %s' % (
            s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
            s.num_peers, s.state, s.name), end='')

    alerts = ses.pop_alerts()
    for a in alerts:
        if a.category() & lt.alert.category_t.error_notification:
            print('\n', a)

    sys.stdout.flush()

    time.sleep(1)

print('\n', h.status().name, 'complete')